In [1]:
%tensorflow_version 2.x

import numpy as np
import pandas as pd
import tensorflow as tf
import re #regex
import keras as keras
from tensorflow.python.keras.preprocessing.text import Tokenizer,text_to_word_sequence
from tensorflow.python.keras.models import load_model, Model,Sequential
from tensorflow.python.keras.layers import Dense, Activation, Dropout, Input, LSTM, Embedding,Flatten
from tensorflow.python.keras.initializers import glorot_uniform,Constant
from tensorflow.python.keras.preprocessing.sequence import pad_sequences 
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.callbacks import ModelCheckpoint,LambdaCallback
from google.colab import files
import random
import sys
import io
import os
import string
from keras.utils import to_categorical
!pip install emoji
import emoji
import re
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
batchsize=32
stepsize = 10
emb_dim=50
fname = "/content/drive/My Drive/Colab Notebooks/Embeddings/glove.6B/glove.6B.50d.txt"
trainPath ="/content/drive/My Drive/Colab Notebooks/Sentiment Classification/train_emoji.csv"

In [3]:
def read_text():
  df=pd.read_csv(trainPath) 
  allText=df.iloc[:, 0].tolist()
  allText = [re.sub('[^a-zA-Z0-9]+', ' ', _).lower() for _ in allText]
  print("allText : ",allText)
  return allText,df

mylist = ["dog", "cat", "wildcat", "thundercat", "cow", "hooo"]
r = re.compile(".*cat")
newlist = list(filter(r.match, mylist)) # Read Note
print(newlist)

def create_tokens(allText,df):
  """****************************************************************************
  Convert the alls sentences into tokenized vector to pass to embedding layer
  ******************************************************************************"""
  tokenizer = Tokenizer(len(allText))
  tokenizer.fit_on_texts(allText)
  sequences = tokenizer.texts_to_sequences(allText) # All the corpus text integer index
  word_index = tokenizer.word_index
  sequences_in = pad_sequences(sequences) #windows before and after the target word.
  sequences_out = to_categorical( df.iloc[:, 1], num_classes=5)  
  maxWords = sequences_in.shape[1]
  vocab_len = len(word_index) + 1  
  print("word_index",word_index)
  print("allText"+str(allText[0]))
  print("sequences  : "+str((sequences)))
  print("text :",(sequences_in[0])," -> sentiment : " ,sequences_out[0])
  print("vocab_len : "+str(vocab_len))
  print("maxWords  : "+str((maxWords)))

  return sequences_in, sequences_out,tokenizer,vocab_len,maxWords,word_index


def get_Embeddings(vocab_len,word_index):
  
  """*****************************************************************************
  Get embedding matrix for all words appering in the training set 
  *******************************************************************************"""
  embeddings_matrix = np.zeros((vocab_len,emb_dim))
  f= open(fname)
  for line in f:
    values = line.split()
    if (values[0] in word_index):
      word = values[0]
      coefs = [float(i) for i in values[1:]]
      if (word in word_index):
        embeddings_matrix[word_index[word],:] = coefs

  f.close()


  return embeddings_matrix


['cat', 'wildcat', 'thundercat']


In [4]:
allText,df = read_text()
sequences_in, sentiment,tokenizer,vocab_len,maxWords,word_index = create_tokens(allText,df)
embeddings_matrix = get_Embeddings(vocab_len,word_index)


allText :  ['i am proud of your achievements', 'it is the worst day in my life', 'miss you so much', 'food is life', 'i love you mum', 'stop saying bullshit', 'congratulations on your acceptance', 'the assignment is too long ', 'i want to go play', 'she did not answer my text ', 'your stupidity has no limit', 'how many points did he score', 'my algorithm performs poorly', 'i got approved', 'stop shouting at me', 'sounds like a fun plan ha ha', 'no one likes him', 'the game just finished', 'i will celebrate soon', 'so sad you are not coming', 'she is my dearest love', 'good job', 'it was funny lol', 'candy is life ', 'the chicago cubs won again', 'i am hungry', 'i am so excited to see you after so long', 'you did well on you exam', 'lets brunch some day', 'he is so cute', 'how dare you ask that', 'do you want to join me for dinner ', 'i said yes', 'she is attractive', 'you suck', 'she smiles a lot', 'he is laughing', 'she takes forever to get ready ', 'french macaroon is so tasty', 'we 

In [5]:
classN=5;

sentence_indices = Input((maxWords,), dtype='int32')
embeddings = Embedding(vocab_len, emb_dim, weights=[embeddings_matrix],name ="emb_layer", trainable=False)(sentence_indices)     
X = LSTM(128,return_sequences=True)(embeddings)
X = Dropout(0.5)(X)
X = LSTM(128,return_sequences=False)(X)
X = Dropout(0.5)(X)
X = Dense(5)(X)
X = Activation('softmax')(X)

# Create Model instance which converts sentence_indices into X.
model = Model(inputs=sentence_indices,outputs=X)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics =['accuracy'])


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
emb_layer (Embedding)        (None, 10, 50)            12950     
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout (Dropout)            (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                

In [6]:
checkpoint_path = "/content/drive/My Drive/Colab Notebooks/Sentiment Classification/sentClassCP.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

model.load_weights(checkpoint_path)
model.fit(sequences_in, sentiment, batch_size=32, epochs=100, shuffle=True, callbacks = [cp_callback])  

Epoch 1/100
5/5 [==============================] - ETA: 0s - loss: 0.0848 - accuracy: 0.9695
Epoch 00001: saving model to /content/drive/My Drive/Colab Notebooks/Sentiment Classification/sentClassCP.ckpt
5/5 [==============================] - 0s 84ms/step - loss: 0.0848 - accuracy: 0.9695
Epoch 2/100
5/5 [==============================] - ETA: 0s - loss: 0.0748 - accuracy: 0.9771
Epoch 00002: saving model to /content/drive/My Drive/Colab Notebooks/Sentiment Classification/sentClassCP.ckpt
5/5 [==============================] - 0s 88ms/step - loss: 0.0748 - accuracy: 0.9771
Epoch 3/100
5/5 [==============================] - ETA: 0s - loss: 0.0732 - accuracy: 0.9847
Epoch 00003: saving model to /content/drive/My Drive/Colab Notebooks/Sentiment Classification/sentClassCP.ckpt
5/5 [==============================] - 0s 83ms/step - loss: 0.0732 - accuracy: 0.9847
Epoch 4/100
5/5 [==============================] - ETA: 0s - loss: 0.0651 - accuracy: 0.9771
Epoch 00004: saving model to /content

In [8]:
emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":smile:",
                    "3": ":disappointed:",
                    "4": ":fork_and_knife:"}

def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji.emojize(emoji_dictionary[str(label)], use_aliases=True)

In [9]:
def sampling():
  testPath ="/content/drive/My Drive/Colab Notebooks/Sentiment Classification/tess.csv"
  df=pd.read_csv(testPath) 
    
  sequences = tokenizer.texts_to_sequences(allText)
  print("sequences : "+str(sequences))
  X_train = pad_sequences(sequences, maxWords)

  # predict a emojy 
  yhat =model.predict(X_train)

  pred = np.argmax(yhat, axis=1)
  for i in range(5):#X.shape[0]
      print(allText[i], label_to_emoji(int(pred[i])))


In [10]:
sampling()

sequences : [[1, 6, 52, 28, 17, 82], [15, 3, 4, 83, 53, 54, 9, 34], [35, 2, 7, 84], [22, 3, 34], [1, 18, 2, 85], [36, 86, 87], [55, 37, 17, 88], [4, 89, 3, 56, 57], [1, 19, 10, 90, 91], [12, 23, 16, 92, 9, 93], [17, 94, 95, 58, 96], [59, 97, 98, 23, 13, 99], [9, 100, 101, 102], [1, 24, 103], [36, 60, 38, 20], [104, 29, 5, 61, 105, 25, 25], [58, 106, 62, 39], [4, 40, 63, 107], [1, 41, 108, 109], [7, 110, 2, 8, 16, 111], [12, 3, 9, 112, 18], [42, 30], [15, 26, 31, 64], [113, 3, 34], [4, 114, 115, 116, 117], [1, 6, 118], [1, 6, 7, 65, 10, 119, 2, 120, 7, 57], [2, 23, 121, 37, 2, 122], [43, 123, 124, 53], [13, 3, 7, 44], [59, 125, 2, 126, 32], [21, 2, 19, 10, 127, 20, 11, 66], [1, 128, 129], [12, 3, 130], [2], [12, 5, 45], [13, 3], [12, 67, 10], [3, 7], [15], [1, 6, 65], [1, 9, 68], [55], [14, 26], [2, 69, 8, 44], [9, 3, 70, 4, 20], [14, 46, 3, 20], [21, 2, 29], [2, 24, 5, 47], [1, 2], [1, 1, 41], [1, 24, 71, 9], [2, 8], [1], [14, 3, 7, 31], [43], [13, 3, 4, 72], [1, 6, 10, 4, 73], [2, 8, 

In [17]:
inp_text = "love does not matter"
text = tokenizer.texts_to_sequences([inp_text])
print(text)
inp = pad_sequences(text, maxWords)
yhat =model.predict(inp)
pred = np.argmax(yhat, axis=1)

print(inp_text, label_to_emoji(int(pred[0])))

[[18, 16]]
love does not matter 😞
